In [24]:
import pandas as pd

contig_count = 8
df = pd.read_csv("assembly_summary.txt",sep="\t",skiprows=1)
df_to_process = df[df['contig_count']==contig_count]
df_to_process


,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
287,GCF_003568645.1,PRJNA224116,SAMN04695901,na,na,142,142,Borrelia turicatae,strain=BTE5EL,na,...,8,8,8,NCBI RefSeq,GCF_003568645.1-RS_2024_03_28,2024/03/28,1249,1157,38,na
382,GCF_027586645.1,PRJNA224116,SAMN31207168,JAOVYB000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_224,na,...,0,8,8,NCBI RefSeq,GCF_027586645.1-RS_2024_04_04,2024/04/04,2885,2806,40,36602320
390,GCF_027586535.1,PRJNA224116,SAMN31207173,JAOVXW000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_234,na,...,0,8,8,NCBI RefSeq,GCF_027586535.1-RS_2024_04_01,2024/04/01,2864,2794,40,36602320
395,GCF_027586855.1,PRJNA224116,SAMN31207154,JAOVYO000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_204,na,...,0,8,8,NCBI RefSeq,GCF_027586855.1-RS_2024_04_02,2024/04/02,2782,2705,40,36602320
430,GCF_027586055.1,PRJNA224116,SAMN31207198,JAOVWX000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_262,na,...,0,8,8,NCBI RefSeq,GCF_027586055.1-RS_2024_04_01,2024/04/01,2921,2847,40,36602320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363987,GCF_040547485.1,PRJNA224116,SAMN41528321,JBEPPT000000000.1,na,3156403,3156403,Agromyces sp. PvR057,strain=PvR057,na,...,0,8,8,NCBI RefSeq,GCF_040547485.1-RS_2024_07_09,2024/07/09,3797,3722,56,na
364236,GCF_040446535.1,PRJNA224116,SAMN41551939,JBEAAK000000000.1,na,3157341,3157341,Pseudoxanthomonas sp. Soil82,strain=Soil82,na,...,0,8,8,NCBI RefSeq,GCF_040446535.1-RS_2024_07_02,2024/07/02,3143,3062,52,na
364353,GCF_040202875.1,PRJNA224116,SAMN40300951,JBEGDW000000000.1,na,3161562,3161562,Xanthomonas sp. WHRI 10204,strain=WHRI 10204,na,...,0,8,8,NCBI RefSeq,GCF_040202875.1-RS_2024_06_17,2024/06/17,4756,4424,235,na
364368,GCF_040253615.1,PRJNA224116,SAMN41772381,JBEJGY000000000.1,na,3162527,3162527,Streptomyces sp. Edi4,strain=Edi4,na,...,0,8,8,NCBI RefSeq,GCF_040253615.1-RS_2024_06_20,2024/06/20,7539,7236,100,na


In [62]:
import pandas as pd
from pathlib import Path
import os
import numpy as np
import subprocess



def find_genomic_fna(row, root_folder):
    # Level 1: Iterate through subfolders
    for subfolder in Path(root_folder).iterdir():
        if subfolder.is_dir():
            # Level 2: Check if "all_assembly_versions" folder exists
            assembly_versions_folder = subfolder / "all_assembly_versions"
            if assembly_versions_folder.exists():
                # Level 3: Iterate through subfolders within "all_assembly_versions"
                for assembly_subfolder in assembly_versions_folder.iterdir():
                    if row.get('#assembly_accession')!=None:
                        if row.get('#assembly_accession') in assembly_subfolder.name:
                        # Level 4: Find file matching criteria
                            for file in assembly_subfolder.glob('*genomic.fna*'):
                                if "cds" not in file.name and "rna" not in file.name:
                                    os.chdir(assembly_subfolder)
                                    #current_directory = os.getcwd()
                                    temp=assembly_subfolder
                                 
                                #Unzip file
                                    if ".gz" in file.name:
                                        return_code1=os.system('gzip -d -k {}'.format(file))
                                        file = str(file)[:-3]
                                        print(file)
                                    else:
                                        return_code1=0
                                    if return_code1==0:
                                        command = ['grep', '-c', '^>', file]
                                        result = subprocess.run(command, capture_output=True, text=True)
                                        num_headers = result.stdout.strip()
                                        print("Num headers:")
                                        print(num_headers)
                                        os.chdir(assembly_subfolder)
                                        current_directory = os.getcwd()
                                        for file in os.listdir(current_directory):
                                            if ".gz" not in file:
                                                try:
                                                    os.unlink(file)
                                                except Exception as e:
                                                    print(e)
                                        return num_headers

df = df_to_process
root_folder = "/Users/niteshku/refseq/refseq/bacteria"

df['num_headers'] = df.apply(lambda row: find_genomic_fna(row, root_folder), axis=1)





/Users/niteshku/refseq/refseq/bacteria/Borrelia_turicatae/all_assembly_versions/GCF_003568645.1_ASM356864v1/GCF_003568645.1_ASM356864v1_genomic.fna
Num headers:
8
/Users/niteshku/refseq/refseq/bacteria/Brachyspira_hyodysenteriae/all_assembly_versions/GCF_027586645.1_ASM2758664v1/GCF_027586645.1_ASM2758664v1_genomic.fna
Num headers:
8
/Users/niteshku/refseq/refseq/bacteria/Brachyspira_hyodysenteriae/all_assembly_versions/GCF_027586535.1_ASM2758653v1/GCF_027586535.1_ASM2758653v1_genomic.fna
Num headers:
8
/Users/niteshku/refseq/refseq/bacteria/Brachyspira_hyodysenteriae/all_assembly_versions/GCF_027586855.1_ASM2758685v1/GCF_027586855.1_ASM2758685v1_genomic.fna
Num headers:
8
/Users/niteshku/refseq/refseq/bacteria/Brachyspira_hyodysenteriae/all_assembly_versions/GCF_027586055.1_ASM2758605v1/GCF_027586055.1_ASM2758605v1_genomic.fna
Num headers:
8
/Users/niteshku/refseq/refseq/bacteria/Brachyspira_hyodysenteriae/all_assembly_versions/GCF_027586045.1_ASM2758604v1/GCF_027586045.1_ASM2758604v1

In [63]:
df


,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id,num_headers
287,GCF_003568645.1,PRJNA224116,SAMN04695901,na,na,142,142,Borrelia turicatae,strain=BTE5EL,na,...,8,8,NCBI RefSeq,GCF_003568645.1-RS_2024_03_28,2024/03/28,1249,1157,38,na,8
382,GCF_027586645.1,PRJNA224116,SAMN31207168,JAOVYB000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_224,na,...,8,8,NCBI RefSeq,GCF_027586645.1-RS_2024_04_04,2024/04/04,2885,2806,40,36602320,8
390,GCF_027586535.1,PRJNA224116,SAMN31207173,JAOVXW000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_234,na,...,8,8,NCBI RefSeq,GCF_027586535.1-RS_2024_04_01,2024/04/01,2864,2794,40,36602320,8
395,GCF_027586855.1,PRJNA224116,SAMN31207154,JAOVYO000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_204,na,...,8,8,NCBI RefSeq,GCF_027586855.1-RS_2024_04_02,2024/04/02,2782,2705,40,36602320,8
430,GCF_027586055.1,PRJNA224116,SAMN31207198,JAOVWX000000000.1,na,159,159,Brachyspira hyodysenteriae,strain=Bhyo_262,na,...,8,8,NCBI RefSeq,GCF_027586055.1-RS_2024_04_01,2024/04/01,2921,2847,40,36602320,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363987,GCF_040547485.1,PRJNA224116,SAMN41528321,JBEPPT000000000.1,na,3156403,3156403,Agromyces sp. PvR057,strain=PvR057,na,...,8,8,NCBI RefSeq,GCF_040547485.1-RS_2024_07_09,2024/07/09,3797,3722,56,na,None
364236,GCF_040446535.1,PRJNA224116,SAMN41551939,JBEAAK000000000.1,na,3157341,3157341,Pseudoxanthomonas sp. Soil82,strain=Soil82,na,...,8,8,NCBI RefSeq,GCF_040446535.1-RS_2024_07_02,2024/07/02,3143,3062,52,na,None
364353,GCF_040202875.1,PRJNA224116,SAMN40300951,JBEGDW000000000.1,na,3161562,3161562,Xanthomonas sp. WHRI 10204,strain=WHRI 10204,na,...,8,8,NCBI RefSeq,GCF_040202875.1-RS_2024_06_17,2024/06/17,4756,4424,235,na,None
364368,GCF_040253615.1,PRJNA224116,SAMN41772381,JBEJGY000000000.1,na,3162527,3162527,Streptomyces sp. Edi4,strain=Edi4,na,...,8,8,NCBI RefSeq,GCF_040253615.1-RS_2024_06_20,2024/06/20,7539,7236,100,na,None


In [64]:
os.chdir('/Users/niteshku/Notebooks/metagenomics')
#df.to_csv("headers.csv")
#headers_todo.to_csv("headers_todo.csv")

In [65]:
df.to_csv("batch8.csv")

In [66]:
#df = pd.read_csv("headers.csv")
df["num_headers"].value_counts()

num_headers
8    960
1     13
7     10
3      9
6      7
4      7
2      6
5      5
Name: count, dtype: int64

In [67]:
selected_rows = df[df['num_headers'].isna()]

In [68]:
selected_rows

,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id,num_headers
2512,GCF_001223585.1,PRJNA224116,SAMEA1401531,CUTT00000000.1,na,197,197,Campylobacter jejuni,strain=OXC6579,na,...,8,8,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/10/05,1686,1583,46,na,None
2518,GCF_001225845.1,PRJNA224116,SAMEA1010475,CUJO00000000.1,na,197,197,Campylobacter jejuni,strain=OXC6306,na,...,8,8,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/10/05,1687,1583,47,na,None
2547,GCF_001234865.1,PRJNA224116,SAMEA1010382,CUHZ00000000.1,na,197,197,Campylobacter jejuni,strain=OXC6268,na,...,8,8,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/10/05,1783,1674,47,na,None
2567,GCF_001231705.1,PRJNA224116,SAMEA1401413,CUVS00000000.1,na,197,197,Campylobacter jejuni,strain=OXC6632,na,...,8,8,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/10/05,1780,1671,46,na,None
2669,GCF_001223725.1,PRJNA224116,SAMEA1010445,CULG00000000.1,na,197,197,Campylobacter jejuni,strain=OXC6346,na,...,8,8,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/10/05,1682,1577,46,na,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363987,GCF_040547485.1,PRJNA224116,SAMN41528321,JBEPPT000000000.1,na,3156403,3156403,Agromyces sp. PvR057,strain=PvR057,na,...,8,8,NCBI RefSeq,GCF_040547485.1-RS_2024_07_09,2024/07/09,3797,3722,56,na,None
364236,GCF_040446535.1,PRJNA224116,SAMN41551939,JBEAAK000000000.1,na,3157341,3157341,Pseudoxanthomonas sp. Soil82,strain=Soil82,na,...,8,8,NCBI RefSeq,GCF_040446535.1-RS_2024_07_02,2024/07/02,3143,3062,52,na,None
364353,GCF_040202875.1,PRJNA224116,SAMN40300951,JBEGDW000000000.1,na,3161562,3161562,Xanthomonas sp. WHRI 10204,strain=WHRI 10204,na,...,8,8,NCBI RefSeq,GCF_040202875.1-RS_2024_06_17,2024/06/17,4756,4424,235,na,None
364368,GCF_040253615.1,PRJNA224116,SAMN41772381,JBEJGY000000000.1,na,3162527,3162527,Streptomyces sp. Edi4,strain=Edi4,na,...,8,8,NCBI RefSeq,GCF_040253615.1-RS_2024_06_20,2024/06/20,7539,7236,100,na,None


In [56]:
a = df[df['num_headers']=="1.0"]

In [17]:
df.astype(str)
headers_todo= df[df['num_headers']!="3"]

In [69]:
import pandas as pd
import os
import ftplib
import subprocess
from urllib.parse import urlparse

def list_ftp_directory(ftp, path):
    file_list = []
    ftp.cwd(path)
    ftp.retrlines('LIST', file_list.append)
    return file_list

def download_ftp_file(ftp, remote_file_path, local_file_path):
    with open(local_file_path, 'wb') as local_file:
        ftp.retrbinary(f"RETR {remote_file_path}", local_file.write)

def process_ftp_directory(ftp_url, local_dir):
    parsed_url = urlparse(ftp_url.replace("https://", "ftp://"))
    ftp_server = parsed_url.netloc
    ftp_path = parsed_url.path
    
    local_folder_path = os.path.join(local_dir, os.path.basename(ftp_path))
    os.makedirs(local_folder_path, exist_ok=True)
    
    with ftplib.FTP(ftp_server) as ftp:
        ftp.login()
        
        file_list = list_ftp_directory(ftp, ftp_path)
        
        for file_info in file_list:
            file_parts = file_info.split()
            file_name = file_parts[-1]
            if file_name == '.' or file_name == '..':
                continue
            if "genomic.fna" in file_name and "cds" not in file_name and "rna" not in file_name:
                remote_file_path = f"{ftp_path}/{file_name}"
                local_file_path = os.path.join(local_folder_path, file_name)
                download_ftp_file(ftp, remote_file_path, local_file_path)
                if ".gz" in local_file_path:
                        return_code1=os.system('gzip -d -k {}'.format(local_file_path))
                        file = str(local_file_path)[:-3]
                        print(file)
                return file  # We are assuming only one relevant file per directory

    return None

def count_fasta_headers(file_path):
    command = ['grep', '-c', '^>', file_path]
    result = subprocess.run(command, capture_output=True, text=True)
    return int(result.stdout.strip())

def process_ftp_paths(df, local_dir):
    num_headers_list = []
    
    for ftp_path in df['ftp_path']:
        try:
            local_file_path = process_ftp_directory(ftp_path, local_dir)
            
            if local_file_path:
                num_headers = count_fasta_headers(local_file_path)
            else:
                num_headers = 0
        except Exception as e:
            print(f"Error processing {ftp_path}: {e}")
            num_headers = 0
        
        num_headers_list.append(num_headers)
    
    df['num_headers'] = num_headers_list
    return df

# Example usage
#data = {'ftp_path': ['https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/417/655/GCF_001417655.1_ASM141765v1']}
df = selected_rows

# Specify the directory where you want to download the files
local_dir = '/Users/niteshku/refseq/refseq/bacteria_test'

# Process the DataFrame
df = process_ftp_paths(df, local_dir)

print(df)


/Users/niteshku/refseq/refseq/bacteria_test/GCF_001223585.1_7213_3_36/GCF_001223585.1_7213_3_36_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_001225845.1_7038_3_56/GCF_001225845.1_7038_3_56_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_001234865.1_7038_3_19/GCF_001234865.1_7038_3_19_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_001231705.1_7213_3_87/GCF_001231705.1_7213_3_87_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_001223725.1_7038_3_96/GCF_001223725.1_7038_3_96_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_001228005.1_7092_1_2/GCF_001228005.1_7092_1_2_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_001232585.1_7065_7_54/GCF_001232585.1_7065_7_54_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_002177655.1_ASM217765v1/GCF_002177655.1_ASM217765v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_019734815.1_ASM1973481v1/GCF_019734815.1_ASM1973481v1_genomic.fna
/Users/niteshku/refs

gzip: data stream error
gzip: /Users/niteshku/refseq/refseq/bacteria_test/GCF_025884095.1_ASM2588409v1/GCF_025884095.1_ASM2588409v1_genomic.fna.gz: uncompress failed


/Users/niteshku/refseq/refseq/bacteria_test/GCF_025884095.1_ASM2588409v1/GCF_025884095.1_ASM2588409v1_genomic.fna
Error processing https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/025/884/095/GCF_025884095.1_ASM2588409v1: invalid literal for int() with base 10: ''
/Users/niteshku/refseq/refseq/bacteria_test/GCF_017161135.1_ASM1716113v1/GCF_017161135.1_ASM1716113v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_903993045.2_AI2856v1_cp/GCF_903993045.2_AI2856v1_cp_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_026614935.1_ASM2661493v1/GCF_026614935.1_ASM2661493v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_947390115.1_17-8104656_assembly/GCF_947390115.1_17-8104656_assembly_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_947390235.1_17-8104516_assembly/GCF_947390235.1_17-8104516_assembly_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_947391185.1_17-8104547_assembly/GCF_947391185.1_17-8104547_assembly_genomic.fna
/Users/niteshku

gzip: data stream error
gzip: /Users/niteshku/refseq/refseq/bacteria_test/GCF_013626015.1_ASM1362601v1/GCF_013626015.1_ASM1362601v1_genomic.fna.gz: uncompress failed


/Users/niteshku/refseq/refseq/bacteria_test/GCF_013626015.1_ASM1362601v1/GCF_013626015.1_ASM1362601v1_genomic.fna
Error processing https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/013/626/015/GCF_013626015.1_ASM1362601v1: invalid literal for int() with base 10: ''
/Users/niteshku/refseq/refseq/bacteria_test/GCF_014269755.1_ASM1426975v1/GCF_014269755.1_ASM1426975v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_014526735.1_ASM1452673v1/GCF_014526735.1_ASM1452673v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_014526745.1_ASM1452674v1/GCF_014526745.1_ASM1452674v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_014526535.1_ASM1452653v1/GCF_014526535.1_ASM1452653v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_900032695.1_12754_4_68/GCF_900032695.1_12754_4_68_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_900026955.1_12754_5_83/GCF_900026955.1_12754_5_83_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_900043355.1_127

gzip: data stream error
gzip: /Users/niteshku/refseq/refseq/bacteria_test/GCF_004106685.1_ASM410668v1/GCF_004106685.1_ASM410668v1_genomic.fna.gz: uncompress failed


/Users/niteshku/refseq/refseq/bacteria_test/GCF_004106685.1_ASM410668v1/GCF_004106685.1_ASM410668v1_genomic.fna
Error processing https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/004/106/685/GCF_004106685.1_ASM410668v1: invalid literal for int() with base 10: ''
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004109865.1_ASM410986v1/GCF_004109865.1_ASM410986v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004111635.1_ASM411163v1/GCF_004111635.1_ASM411163v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004112335.1_ASM411233v1/GCF_004112335.1_ASM411233v1_genomic.fna


gzip: data stream error
gzip: /Users/niteshku/refseq/refseq/bacteria_test/GCF_004113995.1_ASM411399v1/GCF_004113995.1_ASM411399v1_genomic.fna.gz: uncompress failed


/Users/niteshku/refseq/refseq/bacteria_test/GCF_004113995.1_ASM411399v1/GCF_004113995.1_ASM411399v1_genomic.fna
Error processing https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/004/113/995/GCF_004113995.1_ASM411399v1: invalid literal for int() with base 10: ''
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004113565.1_ASM411356v1/GCF_004113565.1_ASM411356v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004113745.1_ASM411374v1/GCF_004113745.1_ASM411374v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004112195.1_ASM411219v1/GCF_004112195.1_ASM411219v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004113575.1_ASM411357v1/GCF_004113575.1_ASM411357v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004108415.1_ASM410841v1/GCF_004108415.1_ASM410841v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004105795.1_ASM410579v1/GCF_004105795.1_ASM410579v1_genomic.fna
/Users/niteshku/refseq/refseq/bacteria_test/GCF_004110795.1_ASM411079v

/var/folders/zk/g8rh78wd4q92gc047lwyptjc0000gp/T/ipykernel_1656/1500022080.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_headers'] = num_headers_list


In [49]:
df.to_csv("batch6_p2.csv")

In [59]:
df

,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id,num_headers
10936,GCF_900455405.1,PRJNA224116,SAMEA3856676,UGUJ00000000.1,na,287,287,Pseudomonas aeruginosa,strain=NCTC11447,na,...,7,7,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/09/14,6381,6203,80,na,7
13216,GCF_040493635.1,PRJNA224116,SAMN35326213,JAULAL000000000.1,na,287,287,Pseudomonas aeruginosa,strain=2105250632,na,...,7,7,NCBI RefSeq,GCF_040493635.1-RS_2024_07_03,2024/07/03,6383,6236,82,na,7
13255,GCF_016622265.1,PRJNA224116,SAMN13075214,WIFS00000000.1,na,287,287,Pseudomonas aeruginosa,strain=KOL14.W.495.35,na,...,7,7,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/12/06,6064,5929,72,na,7
13530,GCF_017693875.1,PRJNA224116,SAMN16132395,JACWHB000000000.1,na,287,287,Pseudomonas aeruginosa,strain=ZBX-P7,na,...,7,7,NCBI RefSeq,GCF_017693875.1-RS_2024_06_03,2024/06/03,6661,6480,79,na,7
15401,GCF_027855975.1,PRJNA224116,SAMN18631017,JAHEQR000000000.1,na,287,287,Pseudomonas aeruginosa,strain=PaLo28,na,...,7,7,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2024/02/24,6376,6175,78,na,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357959,GCF_040409955.1,PRJNA224116,SAMN33330975,na,na,3029062,3029062,Salipiger sp. H15,strain=H15,na,...,7,7,NCBI RefSeq,GCF_040409955.1-RS_2024_06_28,2024/06/28,5848,5708,74,na,7
362440,GCF_040370365.1,PRJNA224116,SAMD00755690,BAABWQ000000000.1,na,3127653,3127653,Shimia sp. NS0008-38b,strain=NS0008-38b,na,...,7,7,NCBI RefSeq,GCF_040370365.1-RS_2024_06_28,2024/06/28,4208,4151,49,na,7
364042,GCF_040049295.1,PRJNA224116,SAMN41518105,JBDZDQ000000000.1,na,3156611,3156611,Sphingomonas sp. KR3-1,strain=KR3-1,na,...,7,7,NCBI RefSeq,GCF_040049295.1-RS_2024_06_05,2024/06/05,4243,4177,55,na,7
364293,GCF_040215725.1,PRJNA224116,SAMN41663692,na,na,3158966,3158966,Rhizobium sp. ZPR4,strain=ZPR4,na,...,7,7,NCBI RefSeq,GCF_040215725.1-RS_2024_06_19,2024/06/19,7198,7040,64,na,7


In [60]:
df.to_csv("batch7_p2.csv")